In [1]:
] dev Photometry

Path `/home/mforets/.julia/dev/Photometry` exists and looks like the correct package. Using existing path.
  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [b4f0291d] ~ LazySets v1.37.9 `~/.julia/dev/LazySets` ⇒ v1.37.10 `~/.julia/dev/LazySets`
  [af68cb61] + Photometry v0.7.2 `~/.julia/dev/Photometry`
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [621f4979] + AbstractFFTs v0.5.0
  [dce04be8] + ArgCheck v2.1.0
  [39de3d68] + AxisArrays v0.4.3
  [198e06fe] + BangBang v0.3.29
  [aafaddc9] + CatIndices v0.2.1
  [c3611d14] + ColorVectorSpace v0.8.6
  [a33af91c] + CompositionsBase v0.1.0
  [ed09eef8] + ComputationalResources v0.3.2
  [150eb455] + CoordinateTransformations v0.6.0
  [dc8bdbbb] + CustomUnitRanges v1.0.0
  [244e2a9f] + DefineSingletons v0.1.0
  [85a47980] + Dictionaries v0.3.2
  [da5c29d0] + EllipsisNotation v0.4.0
  [4f61f5a4] + FFTViews v0.3.1
  [7a1cc6ca] + FFTW v1.2.4
  [f5851436] + FFTW_jll v3.3.9+5
  [a2bd30eb] + Graphics v1.0.2

In [1]:
] st

Status `~/.julia/environments/v1.5/Project.toml`
  [27a7e980] Animations v0.4.0
  [d649c9e1] AssumeGuaranteeContracts v0.1.0 `~/.julia/dev/AssumeGuaranteeContracts`
  [6e4b80f9] BenchmarkTools v0.5.0
  [d3bc4f2e] BranchAndPrune v0.1.0
  [3391f64e] CDDLib v0.6.2
  [336ed68f] CSV v0.7.7
  [9961bab8] Cbc v0.7.0
  [5ae59095] Colors v0.12.4
  [861a8166] Combinatorics v1.0.2
  [a93c6f00] DataFrames v0.21.4
  [ef61062a] DiffEqUncertainty v1.5.0
  [0c46a032] DifferentialEquations v6.15.0
  [31c24e10] Distributions v0.23.4
  [e30172f5] Documenter v0.24.11
  [35a29f4d] DocumenterTools v0.1.7
  [a1e7a1ef] Expokit v0.2.0
  [d4d017d3] ExponentialUtilities v1.8.0 `https://github.com/SciML/ExponentialUtilities.jl.git#master`
  [e2ba6199] ExprTools v0.1.1
  [7a1cc6ca] FFTW v1.2.4
  [fa42c844] FastRounding v0.2.0
  [1a297f60] FillArrays v0.8.14
  [f6369f11] ForwardDiff v0.10.12
  [60bf3e95] GLPK v0.13.0
  [3c7084bd] GLPKMathProgInterface v0.5.0
  [28b8d3ca] GR v0.50.1
  [7073ff75] IJulia v1.21.2
  [621

In [2]:
using Photometry, LazySets, BenchmarkTools, Test, StaticArrays

## Existing code

In [3]:
using Photometry.Aperture: rectangular_overlap_exact,
                           rectangular_overlap_single_subpixel

In [4]:
@testset "overlap - rectangular" begin
    @testset "exact" begin
        @test rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0) ≈ 0.25
    end

    @testset "type stability" begin
        # @inferred rectangular_overlap_exact(0, 0, 1, 1, 2, 2, 0) # TODO area in LazySets.jl not type stable
        @inferred Photometry.Aperture.rectangular_overlap_single_subpixel(0, 0, 1, 1, 2, 2, 0, 5)
    end
end # overlap rectangular

Test Summary:         | Pass  Total
overlap - rectangular |    1      1


Test.DefaultTestSet("overlap - rectangular", Any[Test.DefaultTestSet("exact", Any[], 1, false), Test.DefaultTestSet("type stability", Any[], 0, false)], 0, false)

In [8]:
@test rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0) ≈ 0.25

Test Passed

In [9]:
@test rectangular_overlap_single_subpixel(0, 0, 1, 1, 2, 2, 0, 5) ≈ 1.0

Test Passed

## Inference

Inference fails, because intersection is type unstable: if the intersection is empty, LazySets returns an empty set.

In [10]:
@inferred rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

LoadError: return type Float64 does not match inferred return type Any

In [11]:
@inferred rectangular_overlap_exact(0, 0, 1, 1, 2, 2, 0)

LoadError: return type Float64 does not match inferred return type Any

In [12]:
@inferred rectangular_overlap_single_subpixel(0, 0, 1, 1, 2, 2, 0, 5)

1.0

## Runtime

In [16]:
@btime rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

  7.837 μs (139 allocations: 10.86 KiB)


0.25

In [17]:
@btime rectangular_overlap_exact(0, 0, 1, 1, 2, 2, 0)

  7.770 μs (139 allocations: 10.86 KiB)


1.0

In [18]:
@btime rectangular_overlap_single_subpixel(0, 0, 1, 1, 2, 2, 0, 5)

  152.350 ns (0 allocations: 0 bytes)


1.0

---

## New method

In [33]:
# contrib: https://nbviewer.jupyter.org/github/mforets/escritoire/blob/master/2020/Week11/2D_intersection.ipynb
# the idea is to use the intersection for HPolygon, which is faster than the fallback because the
# normal vectors to the half-spaces are sorted in counter--clockwise fashion
function _intersection_area(X::AbstractHyperrectangle{N},
    Y::LinearMap{N,<:AbstractHyperrectangle{N}}) where {N}
    X_poly = convert(VPolygon, X)
    Y_poly = convert(VPolygon, Y)
    inter = intersection(X_poly, Y_poly)

    return inter isa EmptySet ? 0.0 : area(inter)
end

function _intersection_area_2(X::AbstractHyperrectangle{N},
                              Y::LinearMap{N,<:AbstractHyperrectangle{N}}) where {N}
    
    
    # X in vertex representation
    cX = X.center
    rX = X.radius
    qX = SA[rX[1], -rX[2]]
    Xv = [cX + rX, cX - rX, cX + qX, cX - qX]
    LazySets._four_points_2d!(Xv)

    # Y in vertex representation
    Z = linear_map(Y.M, Y.X)
    Zv = vertices_list(Z)
    LazySets.convex_hull!(Zv)

    Tv = LazySets._intersection_vrep(Xv, Zv)

    if isempty(Tv)
        return 0.0
    else
        return area(Tv)
    end
end

function _intersection_area_3(X::AbstractHyperrectangle{N},
    Y::LinearMap{N,<:AbstractHyperrectangle{N}}) where {N}
    X_poly = convert(VPolygon, X)
    Y_poly = linear_map(Y.M, convert(VPolygon, Y.X))

    Tv = LazySets._intersection_vrep(X_poly.vertices, Y_poly.vertices)
    if isempty(Tv)
        return 0.0
    else
        return LazySets._area_polygon(Tv)
    end
    #inter = intersection(X_poly, Y_poly)
    #return inter isa EmptySet ? 0.0 : area(inter)
end

function _intersection_area_4(X::LazySet{N}, Y::LazySet{N}) where {N}
    vX, vY = vertices_list(X), vertices_list(Y)
    vint = LazySets._intersection_vrep(vX, vY)
    return isempty(vint) ? zero(N) : LazySets._area_polygon(vint)
end

_intersection_area_3 (generic function with 1 method)

In [35]:
using Photometry.Aperture: RotMatrix

function _rectangular_overlap_exact(xmin, ymin, xmax, ymax, w, h, θ)
    R = RotMatrix{2}(deg2rad(θ))
    aper = R * Hyperrectangle(zeros(SVector{2}), SA[w / 2, h / 2])
    dy = ymax - ymin
    dx = xmax - xmin
    pix = Hyperrectangle(SA[dx / 2 + xmin, dy / 2 + ymin], SA[dx / 2, dy / 2])
    return pix, aper
    #return _intersection_area(pix, aper)
end

_rectangular_overlap_exact (generic function with 1 method)

## Inference

In [36]:
X, Y = _rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

(Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.5, 0.5], [0.5, 0.5]), LinearMap{Float64,Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}},Float64,RotMatrix{2,Float64,4}}([1.0 -0.0; 0.0 1.0], Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.0, 0.0], [0.5, 0.5])))

In [37]:
X

Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.5, 0.5], [0.5, 0.5])

In [38]:
Y

LinearMap{Float64,Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}},Float64,RotMatrix{2,Float64,4}}([1.0 -0.0; 0.0 1.0], Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.0, 0.0], [0.5, 0.5]))

In [40]:
@which convert(VPolygon, X)

convert(::Type{VPolygon}, P::AbstractPolytope) in LazySets at /home/mforets/.julia/dev/LazySets/src/convert.jl:128

In [42]:
@btime vertices_list($X)

  51.120 ns (2 allocations: 240 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 0.0]

In [43]:
@which vertices_list(X)

vertices_list(H::AbstractHyperrectangle{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractHyperrectangle.jl:183

In [55]:
typeof(Y)

LinearMap{Float64,Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}},Float64,RotMatrix{2,Float64,4}}

In [56]:
vertices_list(Y)

4-element Array{Array{Float64,1},1}:
 [0.5, 0.5]
 [-0.5, 0.5]
 [-0.5, -0.5]
 [0.5, -0.5]

In [58]:
U = SA[1 2; 0 1] * Hyperrectangle(SA[0, 0.], SA[0.1, 0.1])

LinearMap{Float64,Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}},Int64,SArray{Tuple{2,2},Int64,2,4}}([1 2; 0 1], Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.0, 0.0], [0.1, 0.1]))

In [61]:
vertices_list(U)

vertices_list(lm::LinearMap{N,S,NM,MAT} where MAT<:AbstractArray{NM,2} where NM where S<:LazySet{N}; prune) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/LazyOperations/LinearMap.jl:368

In [65]:
_apply(U) = broadcast(x -> U.M*x, vertices_list(U.X))

_apply (generic function with 1 method)

In [148]:
@btime _apply($U)

  80.682 ns (3 allocations: 384 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [145]:
_apply_gen(U) = broadcast(x -> U.M * x, vertices(U.X))

_apply_gen (generic function with 1 method)

In [147]:
@btime _apply_gen($U)

  109.414 ns (4 allocations: 528 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [69]:
@btime vertices_list($U)

  275.497 ns (9 allocations: 864 bytes)


4-element Array{Array{Float64,1},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [71]:
@which vertices_list(U)

vertices_list(lm::LinearMap{N,S,NM,MAT} where MAT<:AbstractArray{NM,2} where NM where S<:LazySet{N}; prune) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/LazyOperations/LinearMap.jl:368

In [149]:
function _vertices_list(lm::LinearMap{N}; prune::Bool=true) where {N<:Real}
    # collect low-dimensional vertices lists
    vlist_X = vertices_list(lm.X)
    
    vlist = broadcast(x -> lm.M * x, vlist_X)

    # create resulting vertices list
 #=   vlist = Vector{eltype(vlist_X)}()
    sizehint!(vlist, length(vlist_X))
    for v in vlist_X
        push!(vlist, lm.M * v)
    end
=#
    return prune ? convex_hull(vlist) : vlist
    #return vlist
end

function _vertices_list3(lm::LinearMap{N}) where {N}
    vlist_X = vertices_list(lm.X)
    vlist = broadcast(x -> lm.M * x, vlist_X)
    return vlist 
end

function _vertices_list4(lm::LinearMap{N}) where {N}
    return vertices_list(linear_map(lm.M, lm.X))
end

_vertices_list4 (generic function with 1 method)

In [159]:
#vertices_list(linear_map(U.M, U.X))
@btime _vertices_list4($U)

  1.932 μs (33 allocations: 2.00 KiB)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [136]:
@inferred _vertices_list(U)

4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [137]:
@btime _vertices_list($U, prune=false)

  80.356 ns (3 allocations: 384 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [138]:
@btime _vertices_list($U, prune=true)

  118.075 ns (4 allocations: 528 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [130]:
@btime _vertices_list2($U)

  80.521 ns (3 allocations: 384 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [132]:
@btime _vertices_list3($U)

  81.533 ns (3 allocations: 384 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [133]:
@btime _apply($U)

  80.350 ns (3 allocations: 384 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [134]:
U

LinearMap{Float64,Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}},Int64,SArray{Tuple{2,2},Int64,2,4}}([1 2; 0 1], Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.0, 0.0], [0.1, 0.1]))

In [139]:
B = BallInf(SVector{2}(1., 1.), 1.)

convert(Zonotope, B)

Zonotope{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2,2},Float64,2,4}}([1.0, 1.0], [1.0 0.0; 0.0 1.0])

In [140]:
split(B, [1], [1])

2-element Array{Zonotope{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2,2},Float64,2,4}},1}:
 Zonotope{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2,2},Float64,2,4}}([0.5, 1.0], [0.5 0.0; 0.0 1.0])
 Zonotope{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2,2},Float64,2,4}}([1.5, 1.0], [0.5 0.0; 0.0 1.0])

In [109]:
@btime vertices_list($U)

  276.247 ns (9 allocations: 864 bytes)


4-element Array{Array{Float64,1},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [99]:
@inferred _vertices_list(U)

LoadError: return type Array{SArray{Tuple{2},Float64,1,2},1} does not match inferred return type Union{Array{Array{Float64,1},1}, Array{SArray{Tuple{2},Float64,1,2},1}}

In [85]:
@btime _vertices_list($U, prune=true)

  156.068 ns (5 allocations: 544 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.30000000000000004, -0.1]
 [-0.1, -0.1]

In [86]:
@btime _vertices_list($U, prune=false)

  116.013 ns (4 allocations: 400 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.30000000000000004, 0.1]
 [0.1, 0.1]
 [-0.1, -0.1]
 [-0.30000000000000004, -0.1]

In [6]:
a = SA[1 2;3 4.]
b = Hyperrectangle(SA[1., 2], SA[2., 2.])
linear_map(a, b)

Zonotope{Float64,SArray{Tuple{2},Float64,1,2},Array{Float64,2}}([5.0, 11.0], [2.0 4.0; 6.0 8.0])

In [9]:
# linear_map(a, convert(Zonotope, b)) # OK
@which linear_map(a, b)

linear_map(M::AbstractArray{N,2}, Z::AbstractZonotope{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractZonotope.jl:339

In [23]:
@inferred _rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

0.25

In [24]:
@btime _rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

  903.314 ns (28 allocations: 2.78 KiB)


0.25

In [16]:
H = Hyperrectangle(SA[0.5, 0.5], SA[0.5, 0.5])

Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.5, 0.5], [0.5, 0.5])

In [17]:
convert(VPolygon, H)

VPolygon{Float64,SArray{Tuple{2},Float64,1,2}}(SArray{Tuple{2},Float64,1,2}[[1.0, 1.0], [0.0, 1.0], [0.0, 0.0], [1.0, 0.0]])

In [21]:
SA[H.radius[1], -H.radius[2]]

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
  0.5
 -0.5

In [22]:
r = SA[H.radius[1], -H.radius[2]]

points = [H.center + H.radius,
          H.center - H.radius,
          H.center + r,
          H.center - r]

LazySets._four_points_2d!(points)

4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.0]
 [1.0, 0.0]

In [23]:
x = _rectangular_overlap_exact(0, 0, 1, 1, 1, 1, 0)

LoadError: MethodError: no method matching _intersection_vrep(::Array{SArray{Tuple{2},Float64,1,2},1}, ::Array{Array{Float64,1},1})
Closest candidates are:
  _intersection_vrep(::Array{VT,1}, !Matched::Array{VT,1}) where {N, VT<:AbstractArray{N,1}} at /home/mforets/.julia/dev/LazySets/src/Sets/VPolygon.jl:669

In [24]:
Z = linear_map(x.M, x.X)

LoadError: UndefVarError: x not defined

In [25]:
vertices_list(Z)

LoadError: UndefVarError: Z not defined

In [26]:
@inline function _angles(point::AbstractVector{N})  where{N}
    (atand(point[2], point[1]) + 360) % 360
end

_angles (generic function with 1 method)

In [27]:
_angles([1.0, 1.0])

45.0

In [28]:
function _single_quadrant_vertices_enum(G::AbstractMatrix{N}, sorted::Bool=true) where{N}
    if !sorted
        G = sortslices(G, dims=2, by=_angles)
    end
    return VPolygon(2*cumsum(hcat(G, -G), dims=2) .- sum(G, dims=2))
end

function _vertices_list_2D(c::AbstractVector{N}, G::AbstractMatrix{N}; apply_convex_hull::Bool) where {N}
    if apply_convex_hull
        return _vertices_list_iterative(c, G, apply_convex_hull=apply_convex_hull)
    end
    angles = mapslices(_angles, G, dims=1)[1, :]
    perm = sortperm(angles)
    sorted_angles = angles[perm]
    sorted_G = G[:, perm]
    polygons  = Vector{VPolygon{N}}()
    sizehint!(polygons, 4)

    @inbounds for i in zip(0:90:360, 90:90:360)
        index = i[1] .<= sorted_angles .< i[2]
        if sum(index) > 0
            push!(polygons, _single_quadrant_vertices_enum(sorted_G[:, index]))
        end
    end

    return vertices_list(translate(reduce(minkowski_sum, polygons), c))
end

function _vertices_list_2D_v2(c::AbstractVector{N}, G::AbstractMatrix{N}; apply_convex_hull::Bool) where {N}
    if apply_convex_hull
        return _vertices_list_iterative(c, G, apply_convex_hull=apply_convex_hull)
    end
    angles = mapslices(_angles, G, dims=1)[1, :]
    perm = sortperm(angles)
    sorted_angles = angles[perm]
    sorted_G = G[:, perm]
    polygons  = Vector{VPolygon{N}}()
    sizehint!(polygons, 4)

    @inbounds for i in zip(0:90:360, 90:90:360)
        index = i[1] .<= sorted_angles .< i[2]
        if sum(index) > 0
            push!(polygons, _single_quadrant_vertices_enum(sorted_G[:, index]))
        end
    end

    return vertices_list(translate(reduce(minkowski_sum, polygons), c))
end

_vertices_list_2D_v2 (generic function with 1 method)

In [29]:
Y = Zonotope(SA[1, 2.], SA[1 2; 3 4.])
#vertices_list(Y)
#@_vertices_list_2D(Y.center, Y.generators, apply_convex_hull=false)
Z.generators

LoadError: UndefVarError: Z not defined

In [30]:
Z = rand(Zonotope, num_generators=2)
vertices_list(Z)

4-element Array{Array{Float64,1},1}:
 [-0.652669015855391, -3.438102751864682]
 [-0.5977131905886871, -2.7558198838847914]
 [0.7077253854876829, -0.8949415857926888]
 [0.7626812107543869, -0.21265871781279833]

In [31]:
@btime vertices_list($Z)

  4.538 μs (29 allocations: 2.08 KiB)


4-element Array{Array{Float64,1},1}:
 [-0.652669015855391, -3.438102751864682]
 [-0.5977131905886871, -2.7558198838847914]
 [0.7077253854876829, -0.8949415857926888]
 [0.7626812107543869, -0.21265871781279833]

In [41]:
function _vertices_list_iterative_ord1(c::VT, G::MT; apply_convex_hull::Bool) where {N, VT<:AbstractVector{N}, MT}
    vlist = Vector{VT}(undef, 4)
    a = [one(N), one(N)]
    b = [one(N), -one(N)]
    @inbounds begin
        vlist[1] = c .+ G * a
        vlist[2] = c .- G * a
        vlist[3] = c .+ G * b
        vlist[4] = c .- G * b
    end
    return apply_convex_hull ? LazySets._four_points_2d!(vlist) : vlist
end

_vertices_list_iterative_ord1 (generic function with 1 method)

In [62]:
Z = rand(Zonotope, num_generators=2)
@btime vertices_list($Z)

  2.711 μs (45 allocations: 2.91 KiB)


4-element Array{Array{Float64,1},1}:
 [4.710910014978195, 0.5552291684001949]
 [4.20896157428746, 2.9178947335717162]
 [-0.19525542001184615, 2.516543924443947]
 [0.30669302067888915, 0.15387835927242577]

In [63]:
@btime _vertices_list_iterative_ord1($Z.center, $Z.generators, apply_convex_hull=false)

  486.469 ns (11 allocations: 1.05 KiB)


4-element Array{Array{Float64,1},1}:
 [4.710910014978195, 0.5552291684001949]
 [-0.19525542001184615, 2.516543924443947]
 [0.30669302067888915, 0.15387835927242577]
 [4.20896157428746, 2.9178947335717162]

In [65]:
@btime _vertices_list_iterative_ord1($Z.center, $Z.generators, apply_convex_hull=true)

  498.565 ns (11 allocations: 1.05 KiB)


4-element Array{Array{Float64,1},1}:
 [4.710910014978195, 0.5552291684001949]
 [4.20896157428746, 2.9178947335717162]
 [-0.19525542001184615, 2.516543924443947]
 [0.30669302067888915, 0.15387835927242577]

In [74]:
cs = SVector{2}(Z.center)
Gs = SMatrix{2, 2}(Z.generators);

In [75]:
@btime _vertices_list_iterative_ord1($cs, $Gs, apply_convex_hull=false)

  68.916 ns (3 allocations: 336 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [4.710910014978195, 0.5552291684001949]
 [-0.19525542001184615, 2.516543924443947]
 [0.30669302067888915, 0.15387835927242577]
 [4.20896157428746, 2.9178947335717162]

In [76]:
@btime _vertices_list_iterative_ord1($cs, $Gs, apply_convex_hull=true)

  79.388 ns (3 allocations: 336 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [4.710910014978195, 0.5552291684001949]
 [4.20896157428746, 2.9178947335717162]
 [-0.19525542001184615, 2.516543924443947]
 [0.30669302067888915, 0.15387835927242577]

In [66]:
function _vertices_list_iterative_v2(c::VT, G::MT; apply_convex_hull::Bool) where {N, VT<:AbstractVector{N}, MT}
    p = size(G, 2)
    vlist = Vector{VT}()
    sizehint!(vlist, 2^p)

    for ξi in Iterators.product([[1, -1] for i = 1:p]...)
        push!(vlist, c .+ G * collect(ξi))
    end

    return apply_convex_hull ? convex_hull!(vlist) : vlist
end

_vertices_list_iterative_v2 (generic function with 1 method)

In [67]:
@btime _vertices_list_iterative_v2($Z.center, $Z.generators, apply_convex_hull=false)

  2.763 μs (43 allocations: 2.78 KiB)


4-element Array{Array{Float64,1},1}:
 [4.710910014978195, 0.5552291684001949]
 [4.20896157428746, 2.9178947335717162]
 [0.30669302067888915, 0.15387835927242577]
 [-0.19525542001184615, 2.516543924443947]

In [68]:
@btime _vertices_list_iterative_v2($cs, $Gs, apply_convex_hull=false)

  2.273 μs (47 allocations: 2.38 KiB)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.7626812107543869, -0.21265871781279833]
 [-0.5977131905886871, -2.7558198838847914]
 [0.7077253854876829, -0.8949415857926888]
 [-0.652669015855391, -3.438102751864682]

In [72]:
Z.center

2-element Array{Float64,1}:
 2.2578272974831743
 1.535886546422071

In [73]:
cs

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
  0.05500609744949789
 -1.82538073483874

In [ ]:
for ξi in Iterators.product([[1, -1] for i = 1:p]...)
    push!(vlist, c .+ G * collect(ξi))
end

In [77]:
Iterators.product([[1, -1] for i = 1:3]...)

Base.Iterators.ProductIterator{Tuple{Array{Int64,1},Array{Int64,1},Array{Int64,1}}}(([1, -1], [1, -1], [1, -1]))

In [ ]:
@inferred _vertices_list_iterative_ord1($Z.center, $Z.generators, apply_convex_hull=false)

In [36]:
@btime vertices_list($Z)

  4.368 μs (29 allocations: 2.08 KiB)


4-element Array{Array{Float64,1},1}:
 [-0.652669015855391, -3.438102751864682]
 [-0.5977131905886871, -2.7558198838847914]
 [0.7077253854876829, -0.8949415857926888]
 [0.7626812107543869, -0.21265871781279833]

In [117]:
_vertices_list_2D_v2(Z.center, Z.generators, apply_convex_hull=false)

4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.5, 0.5]
 [-0.5, 0.5]
 [-0.5, -0.5]
 [0.5, -0.5]

In [88]:
@btime _vertices_list_2D($Z.center, $Z.generators, apply_convex_hull=false)

  13.510 μs (177 allocations: 9.42 KiB)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.5, 0.5]
 [-0.5, 0.5]
 [-0.5, -0.5]
 [0.5, -0.5]

In [87]:
@btime vertices_list($Z)

  4.307 μs (30 allocations: 2.11 KiB)


4-element Array{Array{Float64,1},1}:
 [-0.5, -0.5]
 [-0.5, 0.5]
 [0.5, -0.5]
 [0.5, 0.5]

In [97]:
Q = rand(Zonotope, num_generators=2)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.7215593048713759, -0.9338053264896249], [-0.10238815732815452 2.427764162585863; -0.1458513979003235 0.4604542027922609])

In [98]:
@btime vertices_list($Q)

  2.882 μs (45 allocations: 2.91 KiB)


4-element Array{Array{Float64,1},1}:
 [3.046935310129084, -0.6192025215976875]
 [3.251711624785393, -0.32749972579704045]
 [-1.6038167003863326, -1.2484081313815623]
 [-1.8085930150426415, -1.5401109271822093]

In [99]:
@btime _vertices_list_2D($Q.center, $Q.generators, apply_convex_hull=false)

  13.631 μs (180 allocations: 9.70 KiB)


4-element Array{Array{Float64,1},1}:
 [3.251711624785393, -0.32749972579704045]
 [-1.6038167003863326, -1.2484081313815623]
 [-1.8085930150426415, -1.5401109271822093]
 [3.046935310129084, -0.6192025215976875]

In [100]:
Q = rand(Zonotope, num_generators=8);

In [101]:
@btime vertices_list($Q)

  187.226 μs (2329 allocations: 249.17 KiB)


16-element Array{Array{Float64,1},1}:
 [-6.263386149395025, -3.345018759744295]
 [-5.066499618143725, -6.8774703400636845]
 [-2.07826736685163, -11.089723225567194]
 [1.7852496725015692, -10.283622679381864]
 [3.858698819312379, -8.327436796000283]
 [5.270017759195234, -6.9282115333318615]
 [8.378499159575602, -2.647214658918098]
 [8.580714458405758, -1.7486771411355106]
 [8.930462213006006, 2.1212633498861853]
 [7.733575681754706, 5.653714930205574]
 [4.745343430462611, 9.865967815709084]
 [0.8818263911094129, 9.059867269523753]
 [-1.1916227557013968, 7.103681386142172]
 [-2.6029416955842515, 5.704456123473751]
 [-5.711423095964619, 1.423459249059988]
 [-5.913638394794777, 0.5249217312774008]

In [102]:
@btime _vertices_list_2D($Q.center, $Q.generators, apply_convex_hull=false)

  29.831 μs (468 allocations: 31.83 KiB)


16-element Array{Array{Float64,1},1}:
 [-6.263386149395025, -3.3450187597442946]
 [-5.066499618143725, -6.8774703400636845]
 [-2.07826736685163, -11.089723225567196]
 [1.7852496725015694, -10.283622679381864]
 [3.858698819312379, -8.327436796000285]
 [5.270017759195234, -6.9282115333318615]
 [8.3784991595756, -2.647214658918098]
 [8.580714458405758, -1.7486771411355102]
 [8.930462213006006, 2.1212633498861857]
 [7.733575681754706, 5.653714930205576]
 [4.745343430462611, 9.865967815709086]
 [0.8818263911094125, 9.059867269523753]
 [-1.1916227557013968, 7.103681386142174]
 [-2.6029416955842515, 5.704456123473751]
 [-5.711423095964619, 1.4234592490599889]
 [-5.913638394794775, 0.5249217312774013]

In [107]:
L = convert(Zonotope, rand(LineSegment))
vertices_list(L)

2-element Array{Array{Float64,1},1}:
 [-0.4463897070499574, 1.1736962963686008]
 [2.363801031434723, -1.0480425969403409]

In [108]:
_vertices_list_2D(L.center, L.generators, apply_convex_hull=false)

2-element Array{Array{Float64,1},1}:
 [-0.4463897070499574, 1.1736962963686008]
 [2.363801031434723, -1.0480425969403409]

In [78]:
h = rand(Hyperrectangle)

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.6136895815123254, -1.0384226596383581], [0.5057834587436363, 1.3112069539243998])

In [79]:
@btime vertices_list($h)

  174.876 ns (7 allocations: 688 bytes)


4-element Array{Array{Float64,1},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, 0.2727842942860417]
 [1.1194730402559618, -2.349629613562758]
 [0.10790612276868916, -2.349629613562758]

In [83]:
cs = SVector{2}(h.center)
rs = SVector{2}(h.radius)
hh = Hyperrectangle(cs, rs)

Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.6136895815123254, -1.0384226596383581], [0.5057834587436363, 1.3112069539243998])

In [84]:
vertices_list(hh)

LoadError: setindex!(::SArray{Tuple{2},Float64,1,2}, value, ::Int) is not defined.

In [99]:
function vertices_list2(H::AbstractHyperrectangle{N}) where {N<:Real}
    n = dim(H)

    # identify flat dimensions and store them in a binary vector whose entry in
    # dimension i is 0 if the radius is zero and 1 otherwise
    # the vector will later also contain entries -1
    trivector = Vector{Int8}(undef, n)
    m = 1
    c = center(H)
    v = MVector{2}(c)
    @inbounds for i in 1:n
        ri = radius_hyperrectangle(H, i)
        if iszero(ri)
            trivector[i] = Int8(0)
        else
            v[i] += ri
            trivector[i] = Int8(1)
            m *= 2
        end
    end

    # create vertices by modifying the three-valued vector and constructing the
    # corresponding point; for efficiency, we create a copy of the old point and
    # modify every entry that has changed in the three-valued vector
    vlist = Vector{typeof(c)}(undef, m)
    vlist[1] = copy(v)
    @inbounds for i in 2:m
        for j in 1:length(v)
            if trivector[j] == Int8(-1)
                trivector[j] = Int8(1)
                v[j] = c[j] + radius_hyperrectangle(H, j)
            elseif trivector[j] == Int8(1)
                trivector[j] = Int8(-1)
                v[j] = c[j] - radius_hyperrectangle(H, j)
                break
            end
        end
        vlist[i] = copy(v)
    end
    return vlist
end

vertices_list2 (generic function with 1 method)

In [100]:
@btime vertices_list2($hh)

  49.907 ns (2 allocations: 240 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, 0.2727842942860417]
 [1.1194730402559618, -2.349629613562758]
 [0.10790612276868916, -2.349629613562758]

In [101]:
@btime vertices_list($h)

  175.153 ns (7 allocations: 688 bytes)


4-element Array{Array{Float64,1},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, 0.2727842942860417]
 [1.1194730402559618, -2.349629613562758]
 [0.10790612276868916, -2.349629613562758]

In [44]:
function _vertices_list_2D_rect(X)
    # X in vertex representation
    cX = X.center
    rX = X.radius
    qX = [rX[1], -rX[2]]
    Xv = [cX + rX, cX - rX, cX + qX, cX - qX]
    LazySets._four_points_2d!(Xv)
end

const NEGx(N) = SA[-one(N) zero(N); zero(N) one(N)]
const NEGy(N) = SA[one(N) zero(N); zero(N) -one(N)]

function _vertices_list_2D_rect_st(X)
    # X in vertex representation
    cX = X.center
    rX = X.radius
    #qX = SA[rX[1], -rX[2]]
    Xv = [cX .+ rX, cX .- rX, cX .+ rX, cX .- rX]
    LazySets._four_points_2d!(Xv)
end

_vertices_list_2D_rect_st (generic function with 1 method)

In [45]:
X

Hyperrectangle{Float64,SArray{Tuple{2},Float64,1,2},SArray{Tuple{2},Float64,1,2}}([0.5, 0.5], [0.5, 0.5])

In [50]:
@btime _vertices_list_2D_rect_st($X)

  118.050 ns (3 allocations: 368 bytes)


2-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.0, 0.0]
 [1.0, 1.0]

In [51]:
@btime vertices_list($X)

  50.694 ns (2 allocations: 240 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.0, 1.0]
 [0.0, 1.0]
 [1.0, 0.0]
 [0.0, 0.0]

In [130]:
@btime _vertices_list_2D_rect_st($hh)

  22.916 ns (1 allocation: 144 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, -2.349629613562758]
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, -2.349629613562758]

In [106]:
@btime _vertices_list_2D_rect($h)

  180.406 ns (6 allocations: 592 bytes)


4-element Array{Array{Float64,1},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, 0.2727842942860417]
 [0.10790612276868916, -2.349629613562758]
 [1.1194730402559618, -2.349629613562758]

In [123]:
@btime _vertices_list_2D_rect_st($hh)

  123.900 ns (3 allocations: 368 bytes)


2-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.10790612276868916, -2.349629613562758]
 [1.1194730402559618, 0.2727842942860417]

In [126]:
@btime _vertices_list_2D_rect_st($hh)

  115.300 ns (3 allocations: 368 bytes)


2-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.10790612276868916, -2.349629613562758]
 [1.1194730402559618, 0.2727842942860417]

In [110]:
@btime _vertices_list_2D_rect($hh)

  60.489 ns (2 allocations: 240 bytes)


4-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [1.1194730402559618, 0.2727842942860417]
 [0.10790612276868916, 0.2727842942860417]
 [0.10790612276868916, -2.349629613562758]
 [1.1194730402559618, -2.349629613562758]